In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import numpy as np
from torchtext.data.utils import get_tokenizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer

2024-02-03 16:29:04.827 Python[57426:3078878] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [ ]:
#### Error with stopword importing, this code fixed it

In [1]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/scottpitcher/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
data = pd.read_csv("data/reddit_vm.csv")
data.head(10)

,title,score,id,url,comms_num,created,body,timestamp
0,Health Canada approves AstraZeneca COVID-19 va...,7,lt74vw,https://www.canadaforums.ca/2021/02/health-can...,0,1.614400e+09,NaN,2021-02-27 06:33:45
1,COVID-19 in Canada: 'Vaccination passports' a ...,2,lsh0ij,https://www.canadaforums.ca/2021/02/covid-19-i...,1,1.614316e+09,NaN,2021-02-26 07:11:07
2,Coronavirus variants could fuel Canada's third...,6,lohlle,https://www.canadaforums.ca/2021/02/coronaviru...,0,1.613887e+09,NaN,2021-02-21 07:50:08
3,Canadian government to extend COVID-19 emergen...,1,lnptv8,https://www.canadaforums.ca/2021/02/canadian-g...,0,1.613796e+09,NaN,2021-02-20 06:35:13
4,Canada: Pfizer is 'extremely committed' to mee...,6,lkslm6,https://www.canadaforums.ca/2021/02/canada-pfi...,0,1.613468e+09,NaN,2021-02-16 11:36:28
5,Canada: Oxford-AstraZeneca vaccine approval ex...,5,lftbji,https://www.canadaforums.ca/2021/02/canada-oxf...,0,1.612869e+09,NaN,2021-02-09 13:17:11
6,Comment,1,ej9x066,NaN,0,1.553474e+09,Your OP. It's not a myth. Only one vaccine con...,2019-03-25 02:34:53
7,Fuck you anti-vaxxing retards,10,g6jkhp,https://www.reddit.com/r/VaccineMyths/comments...,8,1.587663e+09,https://youtu.be/zBkVCpbNnkU,2020-04-23 20:23:42
8,Comment,0,fofa0yy,NaN,0,1.587760e+09,Because Anti-Vaxxers have no sense,2020-04-24 23:19:50
9,Comment,0,ej9xuaf,NaN,0,1.553475e+09,"What do you mean by ""your OP"". I am fairly new...",2019-03-25 02:45:21


In [4]:
data.head(1).body

0    NaN
Name: body, dtype: object

In [5]:
data_full = data.dropna(subset = ["body"])
data_full

,title,score,id,url,comms_num,created,body,timestamp
6,Comment,1,ej9x066,NaN,0,1.553474e+09,Your OP. It's not a myth. Only one vaccine con...,2019-03-25 02:34:53
7,Fuck you anti-vaxxing retards,10,g6jkhp,https://www.reddit.com/r/VaccineMyths/comments...,8,1.587663e+09,https://youtu.be/zBkVCpbNnkU,2020-04-23 20:23:42
8,Comment,0,fofa0yy,NaN,0,1.587760e+09,Because Anti-Vaxxers have no sense,2020-04-24 23:19:50
9,Comment,0,ej9xuaf,NaN,0,1.553475e+09,"What do you mean by ""your OP"". I am fairly new...",2019-03-25 02:45:21
10,Comment,1,ej9x2qr,NaN,0,1.553474e+09,"When they say there's no thimerasol, they mean...",2019-03-25 02:35:47
...,...,...,...,...,...,...,...,...
1597,Comment,2,elrv9ua,NaN,0,1.556223e+09,Lol. My comprehension is comprehensive.\r\nYou...,2019-04-25 23:11:10
1598,Comment,1,elrud7d,NaN,0,1.556223e+09,"If you have trouble finding it, that's one thi...",2019-04-25 23:02:04
1599,Comment,0,elrtjl5,NaN,0,1.556222e+09,Is there music playing where you are because y...,2019-04-25 22:53:50
1600,Comment,1,elrrt5l,NaN,0,1.556221e+09,How long does it take ethyl mercury to metabol...,2019-04-25 22:35:39


In [6]:
descriptions = data_full.body.to_list()
descriptions[:5]

["Your OP. It's not a myth. Only one vaccine contains it and you can get it without it. So your OP is pointless flex. ",
 'https://youtu.be/zBkVCpbNnkU',
 'Because Anti-Vaxxers have no sense',
 'What do you mean by "your OP". I am fairly new to reddit.',
 "When they say there's no thimerasol, they mean in the childhood schedule. That IS a fact. "]

### Looking at word frequencies

In [7]:
threshold = 1000
desc_joined = ''.join(descriptions)
tokens = tokenizer(desc_joined)

freq_dist = FreqDist(tokens)
freq_dist
common_tokens = [token for token in tokens if freq_dist[token] > threshold]
print(common_tokens)

NameError: name 'tokenizer' is not defined

In [ ]:
stop_words = set(stopwords.words("english"))

tokenizer = get_tokenizer("basic_english")
stemmer = PorterStemmer() 

# Complete the function to preprocess sentences
def preprocess_sentences(sentences):
    processed_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        tokens = tokenizer(sentence)
        tokens = [token for token in tokens if token not in stop_words]
        tokens = [stemmer.stem(token) for token in tokens]
        processed_sentences.append(' '.join(tokens))
    return processed_sentences


processed_text = preprocess_sentences(descriptions)
print(processed_text[:5]) 


AttributeError: module 'stopwords' has no attribute 'words'

<1226x7917 sparse matrix of type '<class 'numpy.float64'>'
	with 49039 stored elements in Compressed Sparse Row format>